## Métricas de desempenho

As métricas de desempenho auxiliam a entender como um determinado se comporta ao identificar padrões de um conjunto de dados. Nem sempre identificar apenas a acurácia de um problema é suficiente. Muitas vezes é necessário identificar se a classe crítica de um problema (quando houver) está sendo considerada na escolha de um modelo.

Utilizando o conjunto de dados de câncer de mama, o objetivo deste notebook é avaliar as métricas de desempenho.

In [70]:
from sklearn.datasets import load_breast_cancer
bc = load_breast_cancer()

data = bc.data
target = bc.target

In [71]:
print(data.shape)

(569, 30)


In [72]:
import numpy as np
print(np.unique(target))

[0 1]


Inicialmente, é necessário criar uma divisão dos dados em treino e teste para analisar o desempenho de algum método de classificação através de alguma métrica. Para isso, considere a divisão de 33% para teste e o método de classificação k-NN.

In [73]:
# separando os dados em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=42)

# treinando o modelo 
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

Após o treinamento, as métricas de desempenho são obtidas ao analisar amostra por amostra a relação entre o rótulo conhecido (y_train ou y_test) e o rótulo informado (y_pred) pelo método de classificação. Para identificar a acurácia, basta encontrar a relação de rótulos corretamente predizidos.

In [74]:
# predizendo os rótulos com o modelo
y_pred = knn.predict(X_test)



# calculando a acurácia sem scikit-learn
certos = 0

for idx, rotulo in enumerate(y_pred):
    if rotulo == y_test[idx]:
        certos += 1

print('Acurácia:', certos/y_pred.shape[0])



# calculando a acurácia de forma vetorizada
certos = np.sum(y_test == y_pred)

print('Acurácia:', certos/y_pred.shape[0])



# avaliando o modelo com o scikit-learn
from sklearn.metrics import accuracy_score
print('Acurácia:', accuracy_score(y_test, y_pred))

Acurácia: 0.9308510638297872
Acurácia: 0.9308510638297872
Acurácia: 0.9308510638297872


Algumas informações a mais sobre o desempenho do modelo podem ser extraídas da matriz de confusão, que traça a relação mais precisa dos erros do modelo. Assim, em vez de simplesmente observar quando um rótulo foi corretamente predizido, registra-se também quando houve erros e quais classes foram mais confundidas. 


A matriz de confusão é criada a partir da relação entre os rótulos de referências e os rótulos dados pelo modelo. Um problema de classificação binário, portanto, terá informações a respeito das amostras rotuladas corretamente (verdadeiros positivos e negativos) e das amostras que confundiram o modelo (falsos positivos e negativos).

In [76]:
# predizendo os rótulos a partir do modelo
y_pred = knn.predict(X_test)

# vp = verdadeiros positivos
# vn = verdadeiros negativos
# fp = falsos positivos
# fn = falsos negativos

vp = 0
vn = 0
fp = 0
fn = 0

for pred, true in zip(y_pred, y_test):
    if pred == 1:
        if pred == true:
            vp += 1
        else:
            fp += 1
    else:
        if pred == true:
            vn += 1
        else:
            fn += 1

print('VP:', vp)
print('FP:', fp)
print('VN:', vn)
print('FN:', fn)

print('---')

# calculando a matriz utilizando scikit-learn
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

print('---')

# obtendo os resultados com scikit-learn
vn, fp, fn, vp = confusion_matrix(y_test, y_pred).ravel()
print('VP:', vp)
print('FP:', fp)
print('VN:', vn)
print('FN:', fn)

VP: 114
FP: 6
VN: 61
FN: 7
---
[[ 61   6]
 [  7 114]]
---
VP: 114
FP: 6
VN: 61
FN: 7


A partir dos resultados é possível perceber que o número de verdadeiros positivos e negativos é maior do que as ocorrências de falsos. Isso é um resultado bom para o modelo. As informações dos erros, ou falsos positivos e negativos, ajuda a monitorar como o modelo erra e qual classe ele mais confunde. Em cima disso, decisões podem ser tomadas. Por exemplo, pode-se optar por um modelo que erra mais uma classe do que outra, como em problemas com classes críticas.

No problema abordado nesse notebook, do câncer de mama, existe uma classe crítica. É preferível um modelo que tenha taxa maior de falsos positivos ou falsos negativos? Certamente que os falsos negativos são mais prejudiciais (um laudo que dá resultado negativo para uma pessoa que tem câncer). Uma forma de analisar a qualidade desse modelo considerando a classe crítica é através da revocação.

A revocação encontra a relação de todos verdadeiros positivos com os verdadeiros positivos e falsos negativos. Quanto menos falsos negativos um modelo tiver de resultado, maior será a revocação. No entanto, deve-se tomar cuidado com uma revocação muito alta que gera um modelo inútil. Por exemplo, um modelo que classifica tudo como positivo sempre.

# Exercícios

(1) Seguindo a fórmula dos slides, calcule as medidas de precisão, revocação e f-medida utilizando as informações da matriz de confusão. Se preferir, crie uma função para isso, pois será útil para o exercício seguinte.

In [78]:
"""
p = vp / (vp + fp)
r = vp / (vp + fn)
fm = 2 * (p * r / (p + r))
"""
def prec(vp, fp):
    return vp / (vp+fp)

def recall (vp, fn):
    return vp / (vp+fn)

def fm (p, r):
    return 2 * ((p * r) / (p+r))

p = prec(vp,fp)
r = recall(vp,fn)
fm = fm(p,r)

print (p)
print (r)
print (fm)

0.95
0.9421487603305785
0.946058091286307


(2) Utilizando o conjunto de dados do câncer de mama, avalie os resultados obtidos no teste (em vez do treino, como foi feito até aqui) e verifique se consegue encontrar algum modelo melhor, considerando o conceito de revocação, variando os parâmetros do KNN.

In [82]:
# separando os dados em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=42)

# treinando o modelo 
from sklearn.neighbors import KNeighborsClassifier

for i in range(1,15):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)

    # predizendo os rótulos com o modelo
    y_pred = knn.predict(X_test)

    # calculando a matriz utilizando scikit-learn
    from sklearn.metrics import confusion_matrix
    #print(confusion_matrix(y_test, y_pred))

    vn, fp, fn, vp = confusion_matrix(y_test, y_pred).ravel()

    r = recall(vp,fn)
    best_r = r
    best_n = i

    print(r)

    if r > best_r:
        best_r = r
        best_n = i
        
    

print('\n',best_r)


0.9421487603305785
0.8925619834710744
0.9504132231404959
0.9090909090909091
0.9586776859504132
0.9338842975206612
0.9834710743801653
0.9669421487603306
0.9917355371900827
0.9834710743801653
0.9917355371900827
0.9834710743801653
0.9834710743801653
0.9834710743801653

 0.9834710743801653


In [69]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names = {'classe1','classe2'}))


precision    recall  f1-score   support

     classe1       0.90      0.91      0.90        67
     classe2       0.95      0.94      0.95       121

    accuracy                           0.93       188
   macro avg       0.92      0.93      0.92       188
weighted avg       0.93      0.93      0.93       188



Para conhecimento, depois considere pesquisar a função **classification_report** do Scikit-Learn.